In [1]:
from google.colab import drive
drive.mount('/content/drive')
ROOT='''/content/drive/MyDrive/Colab Notebooks/googledrive/attentionprediction'''

Mounted at /content/drive


In [2]:
import csv
import json
import sqlite3
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from datetime import datetime

In [4]:
def plot_learning_curve_single_subject(marks_conf,counterbalance):

    fig, ax = plt.subplots(figsize=(5, 3))
    rect = patches.Rectangle((0,-0.2), 3, 1.4, linewidth=1,  facecolor='yellow', alpha=0.5)#edgecolor='r',
    conf_mat = marks_conf[9]
    marks_acc = [(conf_mat.diagonal() / (conf_mat.sum(1)+1e-7)).mean()]
    for i in range(1,6):
        conf_mat = marks_conf[(i+1)*8-1] - marks_conf[i*8-1]
        acc = (conf_mat.diagonal() / (conf_mat.sum(1)+1e-7)).mean()
        marks_acc.append(acc)

    ax.scatter(range(len(marks_acc)),marks_acc,c='black',s=60)

    for i in range(len(marks_acc)-1):
        ax.plot([i,i+1],[marks_acc[i],marks_acc[i+1]],c='black')
    plt.yticks([_*0.1 for _ in range(0,11,2)],[round(_*0.1,1) for _ in range(0,11,2)],fontsize=20)
    plt.xticks(range(len(marks_acc)),[_*8 for _ in range(1,len(marks_acc)+1)],fontsize=20)
    plt.suptitle('counterbalance: {}'.format(counterbalance), fontsize=20)
    # Add the patch to the Axes
    ax.add_patch(rect)
    ax.xaxis.set_ticks_position('none')
    ax.yaxis.set_ticks_position('none')
    ax.spines[['right', 'left']].set_visible(False)

def construct_confmat():
    labels = ['h_ivsn','h_gbvs','h_deepgaze']#,'h_catch']
    confmatrices = {**{'h_machine':np.zeros((2,2))}, **{label: np.zeros((1,2)) for label in labels}}
    return confmatrices

def print_metadata(data,dummy_mark='NA'):
    print("====="*20)
    datetime_object1 = datetime.strptime(data[11].split('.')[0], '%Y-%m-%d %H:%M:%S')
    datetime_object2 = datetime.strptime(data[13].split('.')[0], '%Y-%m-%d %H:%M:%S')
    time_used = datetime_object2 - datetime_object1
    time_used = round(time_used.seconds/60,1)
    print("[Finished Date]",datetime_object2)
    print("[Time Used] {} mins".format(time_used))
    print("[Questionnaire]", json.loads(data[-1])["questiondata"])
    print("[Counterbalance]", json.loads(data[-1])["counterbalance"])
    print("[Dummy Test Acc]", dummy_mark)
    return time_used, json.loads(data[-1])["counterbalance"]

conn = sqlite3.connect(ROOT+"/Feedback_FV/ec2_db/turing_feedback.db")
cursor = conn.cursor()
cursor.execute("select * from newdb;")

label_dict = {'human':0,'machine':1,'Human':0,'Machine':1,'ivsn':1,'gbvs':1,'deepgaze':1,'swap':1,'catch':1}
modelname_dict= {'ivsn':'IVSN','gbvs':'GBVS','deepgaze':'DeepGazeIII','chatgpt':"ChatGPT",'swap':'SwappedBackground','catch':'CatchTrial'}

count = 0
participant_count = 0
trials_dict = {}
all_data = cursor.fetchall()
completed_counterbalances = []
dummy_marks_overall = []

confmat_overall = construct_confmat()
catch_trial_mat_overall = np.zeros((1,2))
swap_trial_mat_overall = np.zeros((1,2))
for i in range(len(all_data)):
    if all_data[i][15] in [4]:
#         print(all_data[i][11])
        data_dict = json.loads(all_data[i][17])
        if data_dict['mode'] != 'live':
            continue
        # filter out subjects using dummy trial accuracy
        # original threshold in the paper: 7/12
        dummy_mark = 0
        nTrials = 60
        threshold = nTrials*(7/12) # change to number of total trials
        for dat in data_dict['data']:
            if dat['trialdata']['phase'] == 'TEST' and \
                dat['trialdata']['object response'] == dat['trialdata']['correct object']:
                dummy_mark += 1
        if dummy_mark < threshold:
            continue
        dummy_marks_overall.append(dummy_mark)

        # construct conf matrices
        time_used, counterbalance = print_metadata(all_data[i],"{}/{}".format(dummy_mark,nTrials))
        completed_counterbalances.append(counterbalance)
#         marks = []
#         marks_conf = []
        confmatrices = construct_confmat() # without catch
        catch_trial_mat = np.zeros((1,2))
        swap_trial_mat = np.zeros((1,2))
        marks = []
        marks_conf = []
        participant_count += 1
        nTrial = 0
        for dat in data_dict['data']:
            if dat['trialdata']['phase'] == 'TEST':
                nTrial += 1
                trial_data = {}
                trial_data['workerID'] = data_dict['workerId']
                trial_data['workerData'] = data_dict['questiondata']
                trial_data['trialData'] = dat['trialdata']


#                 trials_dict['Trial_'+str(count)] = trial_data
#                 trial_data = trial_data['trialData']
#                 print(trial_data)
                if trial_data['trialData']['correct response'] == 'Catch':
                    catch_trial_mat[0][label_dict[trial_data['trialData']['choose response']]] += 1
                    catch_trial_mat_overall[0][label_dict[trial_data['trialData']['choose response']]] += 1
                elif trial_data['trialData']['correct response'] == 'Swapped':
                    swap_trial_mat[0][label_dict[trial_data['trialData']['choose response']]] += 1
                    swap_trial_mat_overall[0][label_dict[trial_data['trialData']['choose response']]] += 1
                elif nTrial > 24:
                    confmatrices['h_machine'][label_dict[trial_data['trialData']['correct response']]][label_dict[trial_data['trialData']['choose response']]] += 1
                    confmat_overall['h_machine'][label_dict[trial_data['trialData']['correct response']]][label_dict[trial_data['trialData']['choose response']]] += 1
                    if trial_data['trialData']['correct response'] == "Machine":
                        model = trial_data['trialData']['imageID'].split('/')[-2].split('_')[-1]
                        confmatrices['h_{}'.format(model)][0][label_dict[trial_data['trialData']['choose response']]] += 1
                        confmat_overall['h_{}'.format(model)][0][label_dict[trial_data['trialData']['choose response']]] += 1
                    marks.append(1 if trial_data['trialData']['correct response'] == trial_data['trialData']['choose response'] else 0)
                    marks_conf.append(deepcopy(confmatrices['h_machine']))
#                 else:

#                 else:
#                     print('catch trial')
                # clear inconsistency
                if trial_data['trialData']['correct response']  in ['Catch','Swapped'] or nTrial <=24:
                    continue
                count += 1
                trial_data['trialData']['response_gender'] = trial_data['trialData']['gender response']
                del trial_data['trialData']['gender response']

                trial_data['trialData']['response_speaker'] = trial_data['trialData']['choose response'].lower()
                del trial_data['trialData']['choose response']

                trial_data['trialData']['response_object'] = trial_data['trialData']['object response'].lower()
                del trial_data['trialData']['object response']

                trial_data['trialData']['groundtruth'] = trial_data['trialData']['correct response'].lower()


                trial_data['trialData']['object_groundtruth'] = trial_data['trialData']['correct object'].lower()
                del trial_data['trialData']['correct object']

                if trial_data['trialData']['correct response'] in ["Machine","Catch","Swapped"]:
                    trial_data['trialData']['machine_groundtruth'] = modelname_dict[model.lower()]
                else:
                    trial_data['trialData']['machine_groundtruth'] = ''

                del trial_data['trialData']['correct response']
                trial_data['trialData']["imgtype"] = "naturaldesign"



                trials_dict['Trial_FV_'+str(count)] = trial_data
#         plot_learning_curve_single_subject(marks_conf,participant_count)
        print(confmatrices)
        print('catch',catch_trial_mat)
        print('swap',swap_trial_mat)
#     else:
#         print(all_data[i][15])
conn.close()

[Finished Date] 2025-03-26 08:29:10
[Time Used] 11.5 mins
[Questionnaire] {'country': 'United States', 'age': '46-50', 'engagement': '10', 'difficulty': '4', 'mode': 'live', 'gender': 'female', 'education': 'high school', 'ai-experience': '1', 'condition': 0, 'native': 'yes'}
[Counterbalance] 0
[Dummy Test Acc] 41/60
{'h_machine': array([[9., 3.],
       [5., 7.]]), 'h_ivsn': array([[2., 2.]]), 'h_gbvs': array([[1., 3.]]), 'h_deepgaze': array([[2., 2.]])}
catch [[0. 4.]]
swap [[5. 3.]]
[Finished Date] 2025-03-26 08:32:28
[Time Used] 13.7 mins
[Questionnaire] {'country': 'United States', 'age': '41-45', 'engagement': '4', 'difficulty': '9', 'mode': 'live', 'gender': 'male', 'education': 'master', 'ai-experience': '2', 'condition': 0, 'native': 'yes'}
[Counterbalance] 0
[Dummy Test Acc] 44/60
{'h_machine': array([[11.,  1.],
       [ 8.,  4.]]), 'h_ivsn': array([[4., 0.]]), 'h_gbvs': array([[2., 2.]]), 'h_deepgaze': array([[2., 2.]])}
catch [[0. 4.]]
swap [[3. 5.]]
[Finished Date] 2025-0

In [ ]:
swap_trial_mat_overall,catch_trial_mat_overall

(array([[201., 199.]]), array([[ 37., 163.]]))

Bad pipe message: %s [b'q\xe3/\xb7S\xfd\xc15nIZw\xae5\xeb\x99\xeai\x00\x02\xbc\x00\x00\x00\x01\x00\x02\x00\x03\x00\x04\x00\x05\x00\x06\x00\x07\x00\x08\x00\t\x00\n\x00\x0b\x00\x0c\x00\r\x00\x0e\x00\x0f\x00\x10\x00\x11\x00\x12\x00\x13\x00\x14\x00\x15\x00\x16\x00\x17\x00\x18\x00\x19\x00\x1a\x00\x1b\x00\x1e\x00\x1f\x00 \x00!\x00"\x00#\x00$\x00%\x00&\x00\'\x00(']
Bad pipe message: %s [b'\xa8E\t\xbdL-g\x16\x92\x95\x01qT"3\xf1\xfa\x81\x00\x02\xbc\x00\x00\x00\x01\x00\x02\x00\x03\x00\x04\x00\x05\x00\x06\x00\x07\x00\x08\x00\t\x00\n\x00\x0b\x00\x0c\x00\r\x00\x0e\x00\x0f\x00']
Bad pipe message: %s [b'\x11\x00\x12\x00\x13\x00\x14\x00\x15\x00\x16\x00\x17\x00\x18']
Bad pipe message: %s [b'\x13\xdc\xe7\x91\xde\xafZ|\xf9\x8f\xe7\xa3\xe8\xaf\xe1\x17\xd7\xb4\x00\x02\xbc\x00\x00\x00\x01\x00\x02\x00\x03\x00\x04\x00\x05\x00\x06\x00\x07\x00\x08\x00\t\x00\n\x00\x0b\x00\x0c\x00\r\x00\x0e\x00\x0f\x00\x10\x00\x11\x00\x12\x00\x13\x00\x14\x00\x15\x00\x16\x00\x17\x00\x18\x00\x19\x00\x1a\x00\x1b\x00\x1e\x00\x1f\x00 

In [ ]:
confmat_overall

{'h_machine': array([[403., 197.],
        [260., 340.]]),
 'h_ivsn': array([[115.,  85.]]),
 'h_gbvs': array([[ 95., 105.]]),
 'h_deepgaze': array([[ 50., 150.]])}

In [7]:
np.mean(dummy_marks_overall)/60

np.float64(0.7043333333333333)

In [8]:
len(dummy_marks_overall)

50

In [ ]:
with open("/home/liuxiao/TuringGithubRunnable/attention_prediction_task/Plot/Data_all/humanjudge_FV_Feedback_testphase.json", 'w') as f:
    json.dump(trials_dict,f)


In [ ]:
((403/(403.+ 197.))+105/(95.+ 105.))/2

0.5983333333333334

In [ ]:
(403/(403.+ 197.)),105/(95.+ 105.)

(0.6716666666666666, 0.525)

Bad pipe message: %s [b'\xf1\x96T8q,\x05\xb6\xe7\xa8\x1d\x80}\xb4,^\xae\xc6\x00\x02\xbc\x00\x00\x00\x01\x00\x02\x00\x03\x00\x04\x00\x05\x00\x06\x00\x07\x00\x08\x00\t\x00\n\x00\x0b\x00\x0c\x00\r\x00\x0e\x00\x0f\x00\x10\x00\x11\x00\x12\x00\x13\x00\x14\x00\x15\x00\x16\x00\x17\x00\x18\x00\x19\x00\x1a\x00\x1b\x00\x1e\x00\x1f\x00 \x00!\x00"\x00#\x00$\x00%\x00&\x00\'\x00(\x00)\x00*\x00+\x00,\x00-\x00']
Bad pipe message: %s [b'/\x000\x001\x002\x003\x004\x005\x006\x007\x008\x009\x00:\x00;\x00<\x00=\x00>\x00?\x00@\x00A\x00B\x00C\x00D\x00E']
Bad pipe message: %s [b'\xcdC\xb4y\xfc\xba\xa4a\x1b.\xfb\xbdMb\xaf\x02,t\x00\x02\xbc\x00\x00\x00\x01\x00\x02\x00\x03\x00\x04\x00\x05\x00\x06\x00\x07\x00\x08\x00\t\x00\n\x00\x0b\x00\x0c\x00\r\x00\x0e\x00\x0f\x00\x10\x00\x11\x00\x12\x00\x13\x00\x14\x00\x15\x00\x16\x00\x17\x00\x18\x00\x19\x00\x1a\x00\x1b\x00\x1e\x00\x1f\x00 \x00!\x00"\x00#\x00$\x00%\x00&\x00\'\x00(\x00)\x00*\x00+\x00,\x00-\x00.\x00/\x000\x001\x002\x003\x004\x005\x006\x007\x008\x009\x00:\x00;\x00